# AWS RDS MySQL

In [1]:
import mysql.connector
import os
import pandas as pd
import yaml
from tqdm import tqdm

In [2]:
with open(r"C:\Users\tanch\Desktop\Bot.World\Bot.World\src\main\config\database-config.yaml", "r") as f:
    try:
        config = yaml.safe_load(f)
    except yaml.YAMLError as exc:
        print(exc)
with open(r"C:\Users\tanch\Desktop\Bot.World\Bot.World\src\main\config\seller-config.yaml", "r") as f:
    try:
        config.update(yaml.safe_load(f))
    except yaml.YAMLError as exc:
        print(exc)

In [40]:
cnx = mysql.connector.connect(user=config['DATABASE_USER'], password=config['DATABASE_PASSWORD'],
                              host=config['DATABASE_HOST'], database = config['DATABASE_NAME'],
                              port = 3306)
cursor = cnx.cursor()

In [40]:
# cursor.execute(f"CREATE DATABASE {config['DATABASE_NAME']};")
# cursor.execute("SHOW DATABASES;")
# cursor.execute("DROP database aws_bot_world_mysql;")

### create and import table

In [42]:
data = pd.read_csv(r"C:\Users\tanch\Documents\NTU\NTU Year 4\FYP - GNN\data\shopee-20221117T041452Z-001\shopee\arietes_product_info.csv")
list_of_records = data.to_records(index = False).tolist()

In [43]:
data.head() 

,product_id,product_name,categories,image_url
0,23826146098,Ultrasonic Aroma Humidifier Led light changing,101129 - Home & Living/Home Fragrance & Aromat...,https://cf.shopee.sg/file/sg-11134207-23020-9n...
1,23821255327,Heavy Duty Canvas Tote Bags Stitching Contrast...,100093 - Women Bags/Tote Bags,https://cf.shopee.sg/file/sg-11134207-23020-yu...
2,23821254972,Hot Products EMS Vibration RF V Shape Other Ho...,101661 - Beauty/Beauty Tools/Facial Care Tools...,https://cf.shopee.sg/file/sg-11134207-23020-k1...
3,23821143235,Wart Removal Cream Natural Herbal Skin Care Re...,100427 - Health/Medical Supplies/First Aid Sup...,https://cf.shopee.sg/file/sg-11134207-23020-1u...
4,23720393304,Cute Kids Bag Cartoon Schoolbag Shoulder Bag S...,101744 - Baby & Kids Fashion/Baby & Kids Acces...,https://cf.shopee.sg/file/sg-11134207-23010-t7...


In [20]:
# cursor.execute(f"""DELETE FROM {config['TABLE_NAME']};""")
# cursor.execute(f"""DROP TABLE {config['TABLE_NAME']};""")

In [46]:
cursor.execute(
    f"""CREATE TABLE IF NOT EXISTS {config['PRODUCT_INFO_TABLE_NAME']} (
    product_id BIGINT PRIMARY KEY,
    product_name VARCHAR(255) NOT NULL,
    categories VARCHAR(255) NOT NULL,
    image_url VARCHAR(255) NOT NULL
    );
    """
)

In [48]:
for record in tqdm(list_of_records):
    try:
        cursor.execute(f"""INSERT INTO {config['PRODUCT_INFO_TABLE_NAME']} (product_id, product_name, categories, image_url) VALUES (%s,%s,%s,%s)""", record)
    except mysql.connector.Error as error:
        print(f"Failed to insert record {record}: {error}")  
cnx.commit()

100%|███████████████████████████████████████████████████████████████████████████████| 349/349 [00:02<00:00, 132.84it/s]


### read contents

In [38]:
cursor.execute(f"select * from {config['PRODUCT_INFO_TABLE_NAME']}")

InternalError: Unread result found

In [ ]:
tuple([2952217870, 3233299909, 3295353571])

In [5]:
from pydantic import BaseModel
class Product(BaseModel):
    product_id: int
    product_name: str
    categories: str
    image_url: str

In [48]:
cursor.execute(f"select * from {config['PRODUCT_INFO_TABLE_NAME']} where product_id in {tuple([2952217870, 3233299909, 3295353571])};")


In [52]:
next(cursor)

StopIteration: 

In [ ]:
    cur.execute(f"select * from {database_config['PRODUCT_INFO_TABLE_NAME']} where product_id in {tuple(item_ids)};")


In [13]:
Product(
    *("123",123,123,123)
)

TypeError: __init__() takes exactly 1 positional argument (5 given)

In [ ]:

cur.execute(f"select * from {database_config['PRODUCT_INFO_TABLE_NAME']} where product_id in ();")

cursor

In [79]:
cursor.close()
cnx.close()

InternalError: Unread result found

# SQLITE3

In [1]:
import sqlite3
import pandas as pd

In [2]:
DESIRED_COLUMNS = ['product_id', 'product_name', 'categories', 'image_url']
DATABASE_DIR = r"C:\Users\tanch\Documents\NTU\NTU Year 4\FYP - GNN\Recommender API\deploy-fastapi-recommendation-system\src\database\sqlite3"
DATABASE_NAME = "arietes_product_info"

## read data

In [3]:
data = pd.read_csv(r"C:\Users\tanch\Documents\NTU\NTU Year 4\FYP - GNN\data\shopee-20221117T041452Z-001\shopee\cleaned_transactions.csv")
list_of_records = data[DESIRED_COLUMNS].to_records(index = False).tolist()

In [4]:
data.head()

,product_id,product_name,categories,image_url
0,23826146098,Ultrasonic Aroma Humidifier Led light changing,101129 - Home & Living/Home Fragrance & Aromat...,https://cf.shopee.sg/file/sg-11134207-23020-9n...
1,23821255327,Heavy Duty Canvas Tote Bags Stitching Contrast...,100093 - Women Bags/Tote Bags,https://cf.shopee.sg/file/sg-11134207-23020-yu...
2,23821254972,Hot Products EMS Vibration RF V Shape Other Ho...,101661 - Beauty/Beauty Tools/Facial Care Tools...,https://cf.shopee.sg/file/sg-11134207-23020-k1...
3,23821143235,Wart Removal Cream Natural Herbal Skin Care Re...,100427 - Health/Medical Supplies/First Aid Sup...,https://cf.shopee.sg/file/sg-11134207-23020-1u...
4,23720393304,Cute Kids Bag Cartoon Schoolbag Shoulder Bag S...,101744 - Baby & Kids Fashion/Baby & Kids Acces...,https://cf.shopee.sg/file/sg-11134207-23010-t7...


## Create database
- create table, insert values

In [5]:
con = sqlite3.connect(f"{DATABASE_DIR}/{DATABASE_NAME}.db")
cur = con.cursor()

In [6]:
cur.execute(f"""CREATE TABLE IF NOT EXISTS {DATABASE_NAME}(
                product_id INTEGER NOT NULL PRIMARY KEY, 
                product_name varchar(255), 
                categories varchar(255), 
                image_url varchar(255));""")  

In [7]:
# need to commit sqlite changes
cur.executemany(f'INSERT INTO {DATABASE_NAME} VALUES(?,?,?,?)', list_of_records)
con.commit()

In [11]:
# cur.execute(f"""select * from {DATABASE_NAME}""")

In [10]:
# cur.fetchone()